### Import library and get session

In [1]:
import ace_lib as ace # !pip install first!! & may delete the matplot library
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
import numpy as np
np.set_printoptions(suppress=True)
%matplotlib inline
warnings.filterwarnings("ignore")

s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_QNxQtHrrEKfnYzgGuGW1YE5G




### Create list of alpha what you want to use in population

In [2]:
datasets_df = hf.get_datasets(s) # by default we load all datasets USA TOP3000 delay 1
datasets_df.head() # DataFrame.head() shows first 5 rows of the dataframe 

# select needed datasets
selected_datasets_df = datasets_df.query("""
        delay == 1 &\
        0.8 < coverage <= 1 &\
        0 < fieldCount < 10000 &\
        region == 'USA' &\
        universe == 'TOP3000' &\
        0 < userCount < 1000 &\
        1 < valueScore < 10 &\
        name.str.contains('volatility', case=False) == 1 
    """, engine='python').sort_values(by=['valueScore'], ascending=False)
selected_datasets_df

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount
133,model58,Implied Volatility Data,The dataset provides a standard deviation (vol...,model,model-volatility-models,USA,1,TOP3000,0.9655,None,2.0,59,1314,2
181,option4,Implied Volatility and Pricing for Equity Options,This dataset provides information of options p...,option,option-option,USA,1,TOP3000,0.8218,None,2.0,274,108406,1196
182,option6,Forecasted Volatility for Equity Options,This dataset has various estimates of Options ...,option,option-option-volatility,USA,1,TOP3000,0.8940,None,2.0,189,34450,140
250,pv28,Price Volatility Predictions Data,Forecasted price and volatility data,pv,pv-price-volume,USA,1,TOP3000,0.8039,None,2.0,18,184,6


In [3]:
datafields_df_list = []  # 創建一個空列表來存儲 datafields 資料
id_long = len(selected_datasets_df.id.values.tolist())
for i in range(id_long):
    dataset_id = selected_datasets_df.id.values.tolist()[i]
    datafields_df = hf.get_datafields(s, dataset_id=dataset_id)  # 下載該資料集的所有字段
    datafields_df_list.append(datafields_df)  # 將 datafields 資料附加到列表中
    datafields_df.head()

# 顯示整個 datafields_df_list 中各個資料框的行數
for i, df in enumerate(datafields_df_list):
    print("Rows in datafields_df", i, ":", df.shape[0])

combined_df = pd.concat(datafields_df_list, ignore_index=True) # 將所有資料框整合成一個
print("Rows in combined_df:", combined_df.shape[0]) # 顯示整合後的資料框行數
combined_df.head()

Rows in datafields_df 0 : 2
Rows in datafields_df 1 : 1196
Rows in datafields_df 2 : 140
Rows in datafields_df 3 : 6
Rows in combined_df: 1344


,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount
0,mdl58_securityname,Name of the security,"{'id': 'model58', 'name': 'Implied Volatility ...",model,model-volatility-models,USA,1,TOP3000,MATRIX,0.9654,None,38,388
1,mdl58_volatility,advanced standard deviation of rates in lendin...,"{'id': 'model58', 'name': 'Implied Volatility ...",model,model-volatility-models,USA,1,TOP3000,MATRIX,0.9655,None,50,926
2,opt4_122_call_dis_delta20,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,29,316
3,opt4_122_call_dis_delta25,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,24,92
4,opt4_122_call_dis_delta30,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,12,52


### GeneticAlgorithm

In [4]:
###from geneticalgorithm import GeneticAlgorithm # not finish yet

### Create function which can define population

In [5]:
def generategeneration(df, num_sampindatafield, num_ingeneration):   ###要做出真的只生成固定數量而不是只在datefield上做手腳
    generation_expression_list = [] # ganerate population
    num_sampindatafield = num_sampindatafield # not really population, just a random sample
    operator_m = [ 'ts_scale', 'ts_rank' ]
    for x in random.sample(df.id.values.tolist(), num_sampindatafield):  # 從每個資料框中隨機抽樣 id 值
        for m in operator_m:
            for y in ['5', '21', '63', '126']:
                expression = f"{m}( {x}, {y} )"
                generation_expression_list.append(expression)
    if len(generation_expression_list) <= num_ingeneration:
        random.sample(generation_expression_list, num_ingeneration)
    else:
        return generation_expression_list
    return generation_expression_list

generation_expression_list = generategeneration(df = combined_df ,num_sampindatafield = 20, num_ingeneration = 100)

Father_alpha_list = [ace.generate_alpha( x, region="USA", universe="TOP3000", neutralization="SUBINDUSTRY",) for x in generation_expression_list]

Father_alpha_df = pd.DataFrame(Father_alpha_list)# 將 alpha_list 轉換為 DataFrame
save_path = r'C:\Users\user\ACE2023_v3\database\Father_alpha.csv' # 設定儲存路徑
Father_alpha_df.to_csv(save_path, index=False)  # index=False 表示不寫入索引列 # 將 DataFrame 存儲為 CSV 文件

### Fitness Function

In [6]:
result = ace.simulate_alpha_list_multi(s, Father_alpha_list)
len(result)
result[0].keys()
result[0]['is_stats']
result_performance = hf.prettify_result(result, detailed_tests_view=False)
result_performance.head()

100%|██████████| 54/54 [14:54<00:00, 16.57s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_competition
0,7019006,20000000,1243,1429,0.6080,0.0714,0.0339,0.000235,0.69,2.02,lQELaO7,"ts_scale( opt4_91_call_vola_delta45, 63 )",PASS,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
1,7401114,20000000,1303,1366,0.7134,0.0753,0.0281,0.000211,0.69,2.13,PwxJ5dp,"ts_rank( opt4_91_call_vola_delta45, 63 )",PASS,FAIL,PASS,FAIL,PASS,PASS,PASS,PASS
2,7133710,20000000,1317,1354,1.0077,0.0726,0.0249,0.000144,0.64,2.37,boJY86m,"ts_rank( opt4_122_call_vola_delta80, 63 )",PASS,FAIL,PASS,FAIL,PASS,PASS,PASS,PASS
3,6166003,20000000,1279,1390,0.8235,0.0627,0.0290,0.000152,0.62,2.24,ObR168g,"ts_scale( opt4_365_call_vola_delta45, 21 )",PASS,FAIL,PASS,FAIL,PASS,PASS,PASS,PASS
4,6088426,20000000,1258,1415,0.6265,0.0619,0.0316,0.000198,0.61,1.93,R29J5Aj,"ts_scale( opt4_365_call_vola_delta45, 63 )",PASS,PASS,PASS,FAIL,PASS,PASS,PASS,PASS


In [7]:
save_path = r'C:\Users\user\ACE2023_v3\database\result_performance.csv' # 設定儲存路徑
result_performance.to_csv(save_path, index=False)

In [8]:
# 读取CSV文件并创建DataFrame
data = pd.read_csv('database/result_performance.csv')

filtered_data = data[data['turnover'] < 1.1] # select the suitable turnover "1.1"
survive_fitness_data = filtered_data.nlargest(10, 'fitness')  # select the greatest alpha "10"

save_path = r'C:\Users\user\ACE2023_v3\database\survive_fitness_data.csv'
survive_fitness_data.to_csv(save_path, index=False)

### Crossover & Mutation

In [11]:
before_crossover_data = pd.read_csv('database/survive_fitness_data.csv')

# 获取 'expression' 列的值并添加到 generation_expression_list
generation_expression_list = before_crossover_data['expression'].tolist()

def crossover(expression1, expression2):
    # 拆分表达式为 {m}, {x}, {y} 部分
    parent1 = expression1.split(' ')
    parent2 = expression2.split(' ')
    if random.random() < 0.5:
            son_expression = f"{parent1[0]} {parent2[1]} {parent1[2]}" # new born son
    else:
        son_expression = f"{parent2[0]} {parent1[1]} {parent2[2]}" 
    return son_expression

son_expression_list = [] # new generation_expression_list

# 遍历原始列表中的每个元素
for i in range(0, len(generation_expression_list), 2):
    expression1 = generation_expression_list[i]
    expression2 = generation_expression_list[i + 1]
    
    if random.random() < 0.9: # crossover as "0.9"
        new_expression = crossover(expression1, expression2)
        son_expression_list.append(new_expression)
    else:
        son_expression_list.append(expression1)
        son_expression_list.append(expression2)

In [14]:
def Mutation(expression):
    split = expression.split(' ')
new_generation_expression_list = []

for i in range(len(son_expression_list)):
    if random.random() < 0.1:
        generation_expression_list = generategeneration(df = combined_df ,num_sampindatafield = 100, num_ingeneration = len(son_expression_list))
        nbmuted_expressoin = generation_expression_list[i]
        nbmutedgene = nbmuted_expressoin.split(' ')
        mutagene_expression = generategeneration(df = combined_df)
        mutagene = mutagene_expression.split(' ')
        if random.random() < 0.5:
            muted_expression = f"{mutagene[0]} {nbmutedgene[1]} {mutagene[2]}"
            new_generation_expression_list.append(muted_expression)
        else:
            muted_expression = f"{nbmutedgene[0]} {mutagene[1]} {nbmutedgene[2]}"
            new_generation_expression_list.append(muted_expression)
    else:
        new_generation_expression_list.append(son_expression_list)
        

In [15]:
print(new_generation_expression_list)
save_path = r'C:\Users\user\ACE2023_v3\database\new_generation_expression_list.csv' # 設定儲存路徑
result_performance.to_csv(save_path, index=False)

[['ts_scale( opt4_91_call_vola_delta45, 63 )', 'ts_rank( opt4_91_call_vola_delta45, 63 )', 'ts_scale( opt4_122_call_vola_delta80, 21', 'ts_scale( opt4_365_call_vola_delta45, 63', 'ts_scale( opt4_91_call_vola_delta45, 63', 'ts_rank( opt4_365_call_vola_delta45, 126'], ['ts_scale( opt4_91_call_vola_delta45, 63 )', 'ts_rank( opt4_91_call_vola_delta45, 63 )', 'ts_scale( opt4_122_call_vola_delta80, 21', 'ts_scale( opt4_365_call_vola_delta45, 63', 'ts_scale( opt4_91_call_vola_delta45, 63', 'ts_rank( opt4_365_call_vola_delta45, 126'], ['ts_scale( opt4_91_call_vola_delta45, 63 )', 'ts_rank( opt4_91_call_vola_delta45, 63 )', 'ts_scale( opt4_122_call_vola_delta80, 21', 'ts_scale( opt4_365_call_vola_delta45, 63', 'ts_scale( opt4_91_call_vola_delta45, 63', 'ts_rank( opt4_365_call_vola_delta45, 126'], ['ts_scale( opt4_91_call_vola_delta45, 63 )', 'ts_rank( opt4_91_call_vola_delta45, 63 )', 'ts_scale( opt4_122_call_vola_delta80, 21', 'ts_scale( opt4_365_call_vola_delta45, 63', 'ts_scale( opt4_91_call